In [13]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime,timedelta
import time
from get_histdata import GetHistData_yf
from regret_calc import CalculateReturns,CalculateReturnsList

In [2]:
# Variables for yfinance
look_back=15 #days ago
look_upto=0 #days ago; 0=today
uptrend_period=3 #days
interval='1d' 

to_date=(datetime.today()-timedelta(look_upto)).strftime("%Y-%m-%d")
from_date=(datetime.today()-timedelta(look_back)).strftime("%Y-%m-%d")
# Convert string datetime to datetime datatype
to_date=datetime.strptime(to_date,"%Y-%m-%d")
from_date=datetime.strptime(from_date,"%Y-%m-%d")

In [3]:
data=pd.read_html('https://en.wikipedia.org/wiki/NIFTY_50')[1]
tickers=data['Symbol'].to_list()
tickers=["{}.NS".format(tick) for tick in tickers]
# hist_N50=GetHistData_yf(tickers,from_date,to_date)

In [4]:
# start_time = time.time()
# hist_N50=pd.DataFrame()
# # Get historical data
# print("Getting historical using yfinance")
# print("Fetching data from {} to {}".format(from_date,to_date))

# for i,tick in enumerate(tickers):
#     security=yf.Ticker(tick)
# #     print(i,tick)
# #     print(security.splits)
#     if i == 0 :
#         hist_N50=security.history(start=from_date,end=to_date,interval=interval)
#         hist_N50['Symbol']=tick
#     else :
#         df2=security.history(start=from_date,end=to_date,interval=interval)
#         df2['Symbol']=tick
#         hist_N50=pd.concat([hist_N50,df2])

# print("--- %s seconds ---" % (time.time() - start_time))


In [5]:
def is_consolidating(df,period=15,percentage=2.5):
    recent_closes=df[-period:]['Close']
    max_close=recent_closes.max()
    min_close=recent_closes.min()
    
    if df.shape[0]<period:
        return False
    
    threshold=1-(percentage/100)
    if min_close > (threshold*max_close):
        return True
    return False

def is_breakingout(df,period=15,percentage=2.5):
    last_close=df[-1:]['Close'][0]
    
    if is_consolidating(df[:-1],period,percentage):
        recent_closes=df[-(period+1):-1]['Close']
        if last_close > recent_closes.max():
            return True
    return False

In [6]:
def backtest():
    for i,tick in enumerate(tickers):
        row1=0
        row2=16
        df1=hist_N50[hist_N50['Symbol']==tick]
        for k in range(df1.shape[0]+1):
            df=df1[row1:row2]
            if not df.empty:
    #             print("{} for timeperiod {} to {}".format(tick,df.index[0],df.index[-1]))            
    #             if is_consolidating(df,15,2.5):
    #                 print("for timeperiod {} to {}".format(df.index[0],df.index[-1]))
    #                 print("{} is consolidating.".format(tick))
    #                 print("")
                if is_breakingout(df,10,2.5):
                    print("for timeperiod {} to {}".format(df.index[0],df.index[-1]))
                    print("{} is breakingout!!.".format(tick))
#                     print("")
            row1+=1
            row2+=1
    #     break


In [7]:
def daily_scan(df_in,period=10,pct_chg=2.5):
    '''This functions scans the dataframe with historical values for stocks that are breaking out or consolidating.
    usage: daily_scan(df_in,period=10,pct_chg=2.5)
    returns 2 lists
    ----------------------------
    (df_in, pd.DataFrame)        : A pandas DataFrame containing 'Close' column with closing prices for the required period.
                                   df_in should also have a column 'Symbol' which contains stock tickers.
    (period, int, optional)      : period of scan. default is 10 days.
    (pct_chg, float, optional)   : Percentage change in close prices for the period provided. default is 2.5%.
                                   stock whose close price in past 10 days or the period provided is below 
                                   2.5% of the max close price in this range (consolidating).
    ----------------------------
    returns 2 lists              list 1 = list of consolidating stocks.
                                 list 2 = list of stocks that are breaking out.
    '''
    
    rows=period+5
    cons_stock_list=[]
    breaking=[]
    tickers=df_in['Symbol'].unique()
    for i,tick in enumerate(tickers):
        df1=df_in[df_in['Symbol']==tick]
        df=df1[-rows:]
        if not df.empty:
    #             print("{} for timeperiod {} to {}".format(tick,df.index[0],df.index[-1]))            
            if is_consolidating(df,period,pct_chg):
    #             print("for timeperiod {} to {}".format(df.index[0],df.index[-1]))
#                 print("{} is consolidating.".format(tick))
                cons_stock_list.append(tick)
#                 print("")
            if is_breakingout(df,period,pct_chg):
    #             print("for timeperiod {} to {}".format(df.index[0],df.index[-1]))
#                 print("{} is breakingout!!.".format(tick))
#                 print("")
                breaking.append(tick)
    return cons_stock_list,breaking


In [8]:
daily_scan(GetHistData_yf(tickers,from_date,to_date))

Getting historical using yfinance
Fetching data from 2022-05-29 00:00:00 to 2022-06-13 00:00:00
--- 24.387270212173462 seconds ---


(['HDFCLIFE.NS', 'TCS.NS', 'TATACONSUM.NS'], [])

In [14]:
CalculateReturnsList(['HDFCLIFE.NS'],10000,'2021-10-10')

--- 0.619182825088501 seconds ---


,Symbol,Shares,Entry_date,Exit_date,Entry_price,Exit_price,Invested,Current_val,Days_HODL,Percentage_chg
0,HDFCLIFE.NS,14.0,2021-10-10,2022-06-13,715.3,593.1,10000,8291.54,246,-17.08%


In [ ]:
print(daily_scan.__doc__)